In [1]:
import random
import colorsys

import numpy as np
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from PIL import ImageEnhance
import shutil
import os
import datetime

In [2]:
CAPTCHA_CHARS = list('0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
# Difficult version
CAPTCHA_CHARS_HARD = ['oOQD', '8BE', 'g9q7', '1IiljJ', '2zZ', '5Ss', 'A4', 'mnMN']

def random_color():
    return tuple(map(lambda x: int(x*255), colorsys.hsv_to_rgb(random.random()-0.3, random.randint(8, 10)/10, random.randint(0, 5)/5)))

def random_captcha_str(length):
    return "".join(random.choices(CAPTCHA_CHARS, k=length))

def random_captcha_str_hard(length):
    chars_like = random.choice(CAPTCHA_CHARS_HARD)
    weights_like = np.ones(len(chars_like), dtype=float)
    weights_other = np.zeros(len(CAPTCHA_CHARS), dtype=float) + (1 / len(CAPTCHA_CHARS) * 1.7)
    weights = np.concatenate((weights_like, weights_other), axis=0)
    concat = np.concatenate((list(chars_like), CAPTCHA_CHARS), axis=0)
    return "".join(random.choices(concat, weights=weights, k=length))

random_captcha_str(4), random_captcha_str_hard(4)

('fnSy', '44Ag')

In [3]:
def generate_picture(width=80, height=30):
    image = Image.new('RGB', (width, height), color=(226, 239, 248))
    return image

In [4]:
def draw_captcha(image, captcha, font_size=25, line_count=5):
    draw = ImageDraw.Draw(image)
    # 获取一个font字体对象参数是ttf的字体文件的目录，以及字体的大小
    font_file = os.path.join('./fonts/font3.ttf')
    font = ImageFont.truetype(font_file, size=font_size, index=0)
    for i in range(line_count):
        x1 = random.randint(0, image.width)
        x2 = random.randint(0, image.width)
        y1 = random.randint(0, image.height)
        y2 = random.randint(0, image.height)
        # 随机划线
        draw.line((x1, y1, x2, y2), fill=random_color())
    for i, random_char in enumerate(list(captcha)):
        draw.text((2+i*20, -3), random_char, random_color(), font=font)

    #对比度增强
    enh_con = ImageEnhance.Contrast(image)
    contrast = 1.1
    image = enh_con.enhance(contrast)

    #锐度增强
    enh_sha = ImageEnhance.Sharpness(image)
    sharpness = 3.5
    image = enh_sha.enhance(sharpness)

    return image

draw_captcha(generate_picture(), random_captcha_str_hard(4))

In [9]:
test = os.fspath('../sample/test')
train = os.fspath('../sample/train')

test_1 = os.fspath('../sample/test-1')
train_1 = os.fspath('../sample/train-1')

shutil.rmtree(test)
os.mkdir(test)
shutil.rmtree(train)
os.mkdir(train)

shutil.rmtree(test_1)
os.mkdir(test_1)
shutil.rmtree(train_1)
os.mkdir(train_1)

def gen_4_captcha(_train, _test):
    now_stamp = int(datetime.datetime.now().timestamp())
    for _ in range(10000):
        now_stamp += 1
        label = random_captcha_str_hard(4)
        fname = f'{label}_{now_stamp}.jpg'
        if random.choices([True, False], weights=[7, 3])[0]:
            draw_captcha(generate_picture(), label, line_count=10).save(os.path.join(_train, fname))
        else:
            draw_captcha(generate_picture(), label, line_count=10).save(os.path.join(_test, fname))

def gen_1_captcha(_train, _test):
    now_stamp = int(datetime.datetime.now().timestamp())
    for _ in range(10000):
        now_stamp += 1
        label = random_captcha_str(1)
        fname = f'{label}_{now_stamp}.jpg'
        if random.choices([True, False], weights=[7, 3])[0]:
            draw_captcha(generate_picture(20), label, line_count=4).save(os.path.join(_train, fname))
        else:
            draw_captcha(generate_picture(20), label, line_count=4).save(os.path.join(_test, fname))

In [10]:
gen_4_captcha(train, test)